In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

# Read data from a file
fname = "C:\\Users\\LENOVO\\Desktop\\Computational Physics 2024 feb-jun\\Assignment_solutions\\Assignment4\\Data.txt"
def read_data(filename):
    data = np.loadtxt(filename)
    x = data[:, 1]
    y = data[:, 2]
    yerr = data[:, 3]
    return x, y, yerr

# Define the model: y = ax^2 + bx + c
def model(theta, x):
    a, b, c = theta
    return a * x**2 + b * x + c

# Define the log likelihood
def log_likelihood(theta, x, y, yerr):
    a, b, c = theta
    model_y = model(theta, x)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model_y)**2 / sigma2 + np.log(sigma2))

# Define the log prior
def log_prior(theta):
    a, b, c = theta
    if -10.0 < a < 10.0 and -10.0 < b < 10.0 and -10.0 < c < 10.0:
        return 0.0
    return -np.inf

# Define the log probability
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# Main function to perform MCMC
def perform_mcmc(filename):
    # Read data
    x, y,yerr = read_data(filename)

    # Initial guess for the parameters
    initial = np.array([1.0, 1.0, 1.0])
    nwalkers = 50
    ndim = len(initial)
    nsteps = 4000

    # Set up the sampler
    pos = initial + 1e-4 * np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(x, y, yerr))

    # Run the MCMC
    sampler.run_mcmc(pos, nsteps, progress=True)

    # Plot the chains
    fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
    samples = sampler.get_chain()
    labels = ["a", "b", "c"]
    for i in range(ndim):
        ax = axes[i]
        ax.plot(samples[:, :, i], "k", alpha=0.3)
        ax.set_xlim(0, len(samples))
        ax.set_ylabel(labels[i])
    axes[-1].set_xlabel("step number")
    plt.show()

    # Flatten the chain and discard the burn-in samples
    flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)

    # Use the corner library to visualize the results
    fig = corner.corner(flat_samples, labels=labels)
    plt.show()

    # Get the best-fit parameters and uncertainties
    a_median, b_median, c_median = np.median(flat_samples, axis=0)
    a_sigma, b_sigma, c_sigma = np.std(flat_samples, axis=0)

    print(f"Best-fit values:")
    print(f"a = {a_median:.3f} ± {a_sigma:.3f}")
    print(f"b = {b_median:.3f} ± {b_sigma:.3f}")
    print(f"c = {c_median:.3f} ± {c_sigma:.3f}")

    # Plot the data with the best-fit model and 200 samples from the posterior
    plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
    x_fit = np.linspace(min(x), max(x), 1000)
    for theta in flat_samples[np.random.randint(len(flat_samples), size=200)]:
        plt.plot(x_fit, model(theta, x_fit), color="gray", alpha=0.1)
    plt.plot(x_fit, model([a_median, b_median, c_median], x_fit), color="red", lw=2, label="Best fit")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Data with Best-Fit Model and Posterior Samples")
    plt.legend()
    plt.show()

# Perform MCMC on the provided data file
perform_mcmc(fname)


ModuleNotFoundError: No module named 'emcee'